## Table des Matières

1. [Introduction](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#Introduction)
2. [Étape 1 : Importer les Bibliothèques Nécessaires](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-1-Importer-les-Biblioth%C3%A8ques-N%C3%A9cessaires)
3. [Étape 2 : Définir les Fonctions de Prétraitement et d’Augmentation](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-2-D%C3%A9finir-les-Fonctions-de-Pr%C3%A9traitement-et-d%E2%80%99Augmentation)
4. [Étape 3 : Définir la Fonction de Création des Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-3-D%C3%A9finir-la-Fonction-de-Cr%C3%A9ation-des-Mod%C3%A8les)
5. [Étape 4 : Définir la Fonction de Téléchargement Audio depuis YouTube](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-4-D%C3%A9finir-la-Fonction-de-T%C3%A9l%C3%A9chargement-Audio-depuis-YouTube)
6. [Étape 5 : Initialiser les Hyperparamètres](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-5-Initialiser-les-Hyperparam%C3%A8tres)
7. [Étape 6 : Charger et Prétraiter les Données](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-6-Charger-et-Pr%C3%A9traiter-les-Donn%C3%A9es)
8. [Étape 7 : Itérer sur les Combinaisons d'Hyperparamètres et Entraîner les Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-7-It%C3%A9rer-sur-les-Combinaisons-d'Hyperparam%C3%A8tres-et-Entra%C3%AEner-les-Mod%C3%A8les)
9. [Étape 8 : Analyser et Sauvegarder les Résultats](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-8-Analyser-et-Sauvegarder-les-R%C3%A9sultats)
10. [Étape 9 : Visualiser les Meilleurs Résultats](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-9-Visualiser-les-Meilleurs-R%C3%A9sultats)
11. [Étape 10 : Sauvegarder et Charger les Meilleurs Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-10-Sauvegarder-et-Charger-les-Meilleurs-Mod%C3%A8les)
12. [Étape 11 : Prédiction sur de Nouveaux Échantillons](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-11-Pr%C3%A9diction-sur-de-Nouveaux-%C3%89chantillons)

## Introduction

Dans ce notebook, nous allons :

1. **Charger et prétraiter les données audio** en extrayant les caractéristiques MFCC avec augmentation de données.
2. **Définir et entraîner différents modèles** (CNN 2D, CNN profond, RNN avec LSTM/GRU) en testant diverses combinaisons d'hyperparamètres tels que la taille du batch, le nombre d'époques, le taux d'apprentissage et le nombre de coefficients MFCC.
3. **Évaluer les performances** de chaque modèle et collecter les métriques dans un tableau pour une analyse comparative.
4. **Visualiser les meilleurs résultats** et **sauvegarder les modèles** optimaux pour une utilisation future.

## Étape 1 : Importer les Bibliothèques Nécessaires

Tout d’abord, importons toutes les bibliothèques nécessaires pour le prétraitement des données, la création des modèles, et l’évaluation des performances.

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import joblib
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, recall_score, f1_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, LSTM, GRU, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import yt_dlp as youtube_dl

# Pour afficher les graphiques directement dans le notebook
%matplotlib inline

## Étape 2 : Définir les Fonctions de Prétraitement et d’Augmentation

Nous allons définir les fonctions nécessaires pour :

- **Augmenter les données audio** (ajout de bruit, modification de la hauteur et de la vitesse).
- **Padder ou tronquer les MFCCs** pour assurer une longueur fixe.
- **Extraire les MFCCs** avec augmentation.

In [ ]:
def augment_audio(audio, sr):
    """
    Applique des augmentations sur l'audio : ajout de bruit, modification de la hauteur et de la vitesse.
    """
    # Ajout de bruit
    noise = np.random.randn(len(audio)) * 0.005
    audio_with_noise = audio + noise

    # Modifier la hauteur en fréquence
    try:
        pitch_factor = np.random.uniform(-5, 5)  # En demi-tons
        audio_pitch_shifted = librosa.effects.pitch_shift(audio, sr=sr, n_steps=pitch_factor)
    except:
        audio_pitch_shifted = audio

    # Modifier la vitesse
    try:
        audio_stretched = librosa.effects.time_stretch(audio, rate=np.random.uniform(0.8, 1.2))
    except:
        audio_stretched = audio

    return [audio_with_noise, audio_pitch_shifted, audio_stretched]

def pad_or_truncate_mfcc(mfccs, fixed_length):
    """
    Ajuste la longueur des MFCCs (n_mfcc, T) à (n_mfcc, fixed_length)
    - Tronque si T > fixed_length
    - Ajoute du padding (zéros) si T < fixed_length
    """
    length = mfccs.shape[1]
    if length > fixed_length:
        mfccs = mfccs[:, :fixed_length]
    elif length < fixed_length:
        pad_width = fixed_length - length
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
    return mfccs

def extract_features_with_augmentation(file_path, fixed_length, n_mfcc=40):
    """
    Extrait les MFCCs d'un fichier audio avec augmentation de données.
    """
    try:
        audio, sr = librosa.load(file_path, duration=30)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs = pad_or_truncate_mfcc(mfccs, fixed_length)

        # Augmentations
        augmented_audios = augment_audio(audio, sr)
        augmented_mfccs = []
        for a in augmented_audios:
            m = librosa.feature.mfcc(y=a, sr=sr, n_mfcc=n_mfcc)
            m = pad_or_truncate_mfcc(m, fixed_length)
            augmented_mfccs.append(m)

        # On renvoie la liste (original + augmentations) directement sous forme (n_mfcc, T)
        all_mfccs = [mfccs] + augmented_mfccs
        return all_mfccs
    except Exception as e:
        print(f"Erreur lors du traitement du fichier {file_path}: {e}")
        return []

## Étape 3 : Définir la Fonction de Création des Modèles

Nous allons définir une fonction qui crée différents types de modèles en fonction des hyperparamètres spécifiés.

In [ ]:
def create_model(model_type, input_shape, num_classes, learning_rate=0.001):
    """
    Crée un modèle de deep learning en fonction du type spécifié.
    """
    model = Sequential()
    
    if model_type == 'CNN_2D_basic':
        # Modèle CNN 2D de base avec padding 'same'
        model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
    
    elif model_type == 'CNN_2D_deep':
        # Modèle CNN 2D plus profond avec padding 'same'
        model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(num_classes, activation='softmax'))
    
    elif model_type == 'CRNN':
        # Modèle Convolutional Recurrent Neural Network avec padding 'same'
        model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(64))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
    
    elif model_type == 'CNN_1D':
        # Modèle CNN 1D avec padding 'same'
        model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
    
    elif model_type == 'RNN_LSTM':
        # Modèle avec couches LSTM
        # Reshape pour LSTM : (N, T, n_mfcc)
        model.add(Flatten(input_shape=input_shape))  # Convertir en séquence pour LSTM
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(LSTM(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(64))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
    
    elif model_type == 'RNN_GRU':
        # Modèle avec couches GRU
        # Reshape pour GRU : (N, T, n_mfcc)
        model.add(Flatten(input_shape=input_shape))  # Convertir en séquence pour GRU
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(GRU(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(GRU(64))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))
    
    else:
        raise ValueError(f"Type de modèle inconnu: {model_type}")

    # Compilation du modèle
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


## Étape 4 : Définir la Fonction de Téléchargement Audio depuis YouTube

Cette fonction permet de télécharger un fichier audio depuis une URL YouTube pour tester les prédictions.

In [ ]:
def download_youtube_audio(youtube_url, output_path="downloaded_audio.wav"):
    """
    Télécharge l'audio d'une vidéo YouTube et le sauvegarde en fichier WAV.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }

    # Télécharger l'audio
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    # Renommer le fichier pour l'utiliser
    if os.path.exists("temp_audio.wav"):
        os.rename("temp_audio.wav", output_path)
        print(f"Audio téléchargé et sauvegardé sous : {output_path}")
    else:
        print("Erreur lors du téléchargement.")


## Étape 5 : Initialiser les Hyperparamètres

Définissons les différentes valeurs des hyperparamètres que nous allons tester.

In [ ]:
# Définition des hyperparamètres à tester
batch_sizes = [16, 32, 64]
epochs_list = [20, 50, 100]
learning_rates = [0.001, 0.0001, 0.00001]
model_types = ['CNN_2D_basic', 'CNN_2D_deep', 'CRNN', 'RNN_LSTM', 'RNN_GRU']  # Ajouter d'autres modèles si nécessaire
n_mfcc_values = [20, 40, 60]  # "token size" interprété comme nombre de MFCCs

# Préparation du tableau de résultats
results = []

# Créer un répertoire pour sauvegarder les modèles
os.makedirs('models', exist_ok=True)


## Étape 6 : Charger et Prétraiter les Données

Pour chaque valeur de `n_mfcc`, nous chargerons et prétraiterons les données audio avec augmentation.

In [ ]:
def load_audio_features_with_augmentation(data_path, fixed_length, n_mfcc=40):
    """
    Charge les features audio avec augmentation et ajuste les MFCCs à une longueur fixe.
    """
    genres = os.listdir(data_path)
    X_list = []
    y_list = []

    for genre in genres:
        genre_path = os.path.join(data_path, genre)
        for file_name in os.listdir(genre_path):
            file_path = os.path.join(genre_path, file_name)
            mfccs_list = extract_features_with_augmentation(file_path, fixed_length=fixed_length, n_mfcc=n_mfcc)
            for m in mfccs_list:
                # m est de forme (n_mfcc, fixed_length)
                X_list.append(m)
                y_list.append(genre)

    X = np.array(X_list)  # X: (N, n_mfcc, fixed_length)
    y = np.array(y_list)

    # Ajout de la dimension "canal" pour le CNN 2D : (N, n_mfcc, fixed_length, 1)
    X = X[..., np.newaxis]
    return X, y


## Étape 7 : Itérer sur les Combinaisons d'Hyperparamètres et Entraîner les Modèles

Nous allons maintenant itérer sur toutes les combinaisons possibles d'hyperparamètres et de types de modèles, entraîner chaque modèle, évaluer ses performances, et enregistrer les résultats.

In [ ]:
def model_exists(n_mfcc, model_type, batch_size, epochs, lr):
    """
    Vérifie si le modèle avec les hyperparamètres spécifiés existe déjà.
    """
    model_save_path = f"models/n_mfcc_{n_mfcc}/{model_type}_bs{batch_size}_ep{epochs}_lr{lr}.keras"
    return os.path.exists(model_save_path)

In [ ]:
# Chemin vers les données audio
data_path = "Data/genres_original"

# Définir fixed_length en fonction de n_mfcc et des paramètres de vos MFCC
fixed_length = 1293  # À ajuster selon votre dataset (en fonction du sr et hop_length)

# Itération sur les différentes valeurs de n_mfcc (token size)
for n_mfcc in n_mfcc_values:
    print(f"\nTraitement pour n_mfcc = {n_mfcc}")
    
    # Charger les données avec augmentation
    X, y = load_audio_features_with_augmentation(data_path, fixed_length, n_mfcc=n_mfcc)

    print(f"Shape des données: X={X.shape}, y={y.shape}")
    
    # Encodage des labels
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded, num_classes=len(encoder.classes_))

    # Sauvegarder l'encodeur (écrasez le précédent si nécessaire)
    joblib.dump(encoder, 'label_encoder.joblib')

    # Split des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

    # Normalisation des données
    # Par exemple, vous pouvez normaliser les MFCCs entre 0 et 1
    X_train = X_train / np.max(X_train)
    X_test = X_test / np.max(X_test)

    # Itération sur les différentes combinaisons d'hyperparamètres
    for model_type, batch_size, epochs, lr in itertools.product(model_types, batch_sizes, epochs_list, learning_rates):
        print(f"Entraînement du modèle: {model_type}, Batch Size: {batch_size}, Epochs: {epochs}, Learning Rate: {lr}")
        
        # Définir le chemin de sauvegarde du modèle
        model_save_path = f"models/n_mfcc_{n_mfcc}/{model_type}_bs{batch_size}_ep{epochs}_lr{lr}.keras"
        
        # Vérifier si le modèle existe déjà
        if model_exists(n_mfcc, model_type, batch_size, epochs, lr):
            print(f"Le modèle {model_type} avec bs={batch_size}, ep={epochs}, lr={lr} existe déjà. Passage au suivant.")
            continue  # Passer à la prochaine combinaison
        
        # Créer le répertoire si nécessaire
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
        
        # Définir le callback EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        # Définir un ModelCheckpoint pour sauvegarder le meilleur modèle
        checkpoint = ModelCheckpoint(model_save_path, monitor='val_accuracy', save_best_only=True, verbose=1)
        
        # Définir le modèle
        input_shape = X_train.shape[1:]  # (n_mfcc, fixed_length, 1)
        num_classes = len(encoder.classes_)
        model = create_model(model_type, input_shape, num_classes, learning_rate=lr)

        # Entraîner le modèle
        history = model.fit(
            X_train, y_train, 
            validation_split=0.2, 
            epochs=epochs, 
            batch_size=batch_size, 
            callbacks=[early_stopping, checkpoint], 
            verbose=1  # Vous pouvez changer en 0 pour moins de sorties
        )

        # Évaluer le modèle
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        val_loss = min(history.history['val_loss'])
        val_accuracy = max(history.history['val_accuracy'])

        # Prédictions pour les métriques supplémentaires
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_true = np.argmax(y_test, axis=-1)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

        # Enregistrer les résultats
        result = {
            'n_mfcc': n_mfcc,
            'model_type': model_type,
            'batch_size': batch_size,
            'epochs': epochs,
            'learning_rate': lr,
            'accuracy': accuracy,
            'loss': loss,
            'validation_accuracy': val_accuracy,
            'validation_loss': val_loss,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }
        results.append(result)
        print(f"Résultat: {result}")

        # Sauvegarder les résultats après chaque entraînement pour éviter les pertes en cas d'interruption
        results_df = pd.DataFrame(results)
        results_df.to_csv('model_results.csv', index=False)
        print("Résultats sauvegardés dans 'model_results.csv'.")

**Explications :**

- **Itération sur `n_mfcc`** : Pour chaque valeur de `n_mfcc` (nombre de coefficients MFCC), nous chargeons et prétraitons les données.
- **Normalisation** : Les MFCCs sont normalisés en divisant par leur maximum pour faciliter la convergence du modèle.
- **Boucle Imbriquée** : Utilisation de `itertools.product` pour parcourir toutes les combinaisons possibles des hyperparamètres et des types de modèles.
- **Métriques Supplémentaires** : En plus de la précision et de la perte, nous calculons également la précision (`precision_score`), le rappel (`recall_score`) et le score F1 (`f1_score`) pour une évaluation plus complète.
- **Sauvegarde des Modèles** : Les meilleurs modèles basés sur la précision de validation sont sauvegardés grâce au `ModelCheckpoint`.

## Étape 8 : Analyser et Sauvegarder les Résultats

Après avoir entraîné tous les modèles, nous allons collecter les résultats dans un tableau et les sauvegarder dans un fichier CSV pour une analyse ultérieure.

In [ ]:
# Convertir les résultats en DataFrame et sauvegarder
results_df = pd.DataFrame(results)
results_df.to_csv('model_results.csv', index=False)
print("\nTous les résultats ont été sauvegardés dans 'model_results.csv'.")


## Étape 9 : Visualiser les Meilleurs Résultats

Nous allons afficher les 10 meilleures configurations basées sur la précision de test.

In [ ]:
# Afficher les meilleurs résultats
best_results = results_df.sort_values(by='accuracy', ascending=False).head(10)
print("\nTop 10 des meilleures configurations :")
display(best_results)

### Visualisation des Résultats

Pour mieux comprendre les performances des différents modèles et hyperparamètres, nous pouvons créer des visualisations.

In [ ]:
# Heatmap de la précision en fonction du modèle et du taux d'apprentissage pour chaque n_mfcc
for n_mfcc in n_mfcc_values:
    plt.figure(figsize=(12, 8))
    subset = results_df[results_df['n_mfcc'] == n_mfcc]
    pivot_table = subset.pivot_table(values='accuracy', index='model_type', columns='learning_rate', aggfunc='mean')
    sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
    plt.title(f"Précision moyenne par modèle et taux d'apprentissage (n_mfcc={n_mfcc})")
    plt.ylabel("Type de Modèle")
    plt.xlabel("Taux d'Apprentissage")
    plt.show()


In [ ]:
# Boxplot de la précision par type de modèle
plt.figure(figsize=(14, 8))
sns.boxplot(x='model_type', y='accuracy', data=results_df)
plt.title("Distribution de la Précision par Type de Modèle")
plt.xlabel("Type de Modèle")
plt.ylabel("Précision")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Scatter plot de la précision en fonction du taux d'apprentissage pour chaque modèle
plt.figure(figsize=(14, 8))
sns.scatterplot(x='learning_rate', y='accuracy', hue='model_type', style='n_mfcc', data=results_df)
plt.xscale('log')
plt.title("Précision en fonction du Taux d'Apprentissage par Type de Modèle")
plt.xlabel("Taux d'Apprentissage (échelle logarithmique)")
plt.ylabel("Précision")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
# Heatmap de la perte de validation par modèle et taux d'apprentissage pour chaque n_mfcc
for n_mfcc in n_mfcc_values:
    plt.figure(figsize=(12, 8))
    subset = results_df[results_df['n_mfcc'] == n_mfcc]
    pivot_table = subset.pivot_table(values='validation_loss', index='model_type', columns='learning_rate', aggfunc='mean')
    sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='magma')
    plt.title(f"Perte de Validation moyenne par modèle et taux d'apprentissage (n_mfcc={n_mfcc})")
    plt.ylabel("Type de Modèle")
    plt.xlabel("Taux d'Apprentissage")
    plt.show()

**Explications :**

- **Heatmap de la Précision** : Compare la précision moyenne pour chaque type de modèle et taux d'apprentissage, segmenté par `n_mfcc`.
- **Boxplot de la Précision** : Montre la distribution de la précision pour chaque type de modèle, permettant d'identifier la variabilité et les performances médianes.
- **Scatter Plot de la Précision** : Visualise la relation entre le taux d'apprentissage et la précision, avec une distinction par type de modèle et `n_mfcc`.
- **Boxplot du Score F1** : Permet de visualiser la distribution du score F1, une métrique équilibrée entre précision et rappel.
- **Heatmap de la Perte de Validation** : Compare la perte de validation moyenne pour chaque type de modèle et taux d'apprentissage, segmenté par `n_mfcc`.

## Étape 10 : Évaluation Approfondie

Pour une évaluation plus complète, calculons des métriques supplémentaires telles que le rapport de classification (classification report) et les courbes ROC (si applicable).

### 10.1. Rapport de Classification

Le rapport de classification fournit la précision, le rappel et le score F1 pour chaque classe.

In [ ]:
# Sélectionner le meilleur modèle basé sur la précision
best_result = results_df.sort_values(by='accuracy', ascending=False).iloc[0]
best_model_path = f"models/n_mfcc_{best_result['n_mfcc']}/{best_result['model_type']}_bs{best_result['batch_size']}_ep{best_result['epochs']}_lr{best_result['learning_rate']}.keras"
best_model = load_model(best_model_path)
print(f"\nChargement du meilleur modèle : {best_model_path}")

# Re-évaluer pour obtenir le rapport de classification
y_pred = np.argmax(best_model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

print("\nRapport de Classification pour le Meilleur Modèle :")
print(classification_report(y_true, y_pred, target_names=encoder.classes_, zero_division=0))


### 10.2. Courbes ROC et AUC

Pour chaque classe, nous pouvons tracer les courbes ROC et calculer l'AUC. Cependant, cela nécessite des prédictions en probabilités et une binarisation des labels.

**Note** : Les courbes ROC sont plus pertinentes pour des problèmes de classification binaire. Pour des classes multiples, nous utilisons une approche "un contre tous".

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# Binariser les labels
y_true_binarized = label_binarize(y_true, classes=np.arange(len(encoder.classes_)))
y_pred_proba = best_model.predict(X_test)

# Calculer les courbes ROC et AUC pour chaque classe
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = y_true_binarized.shape[1]

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Tracer toutes les courbes ROC
plt.figure(figsize=(14, 10))
colors = sns.color_palette("hls", n_classes)
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(encoder.classes_[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de Faux Positifs')
plt.ylabel('Taux de Vrais Positifs')
plt.title('Courbes ROC Multiclasses pour le Meilleur Modèle')
plt.legend(loc="lower right")
plt.show()


**Explications :**

- **Rapport de Classification** : Fournit une vue détaillée des performances du modèle par classe, y compris la précision, le rappel et le score F1.
- **Courbes ROC et AUC** : Mesurent la capacité du modèle à distinguer chaque classe par rapport aux autres. L'AUC (Area Under the Curve) est une mesure globale de performance pour chaque classe.

### 10.3. Matrice de Confusion

Bien que vous ayez déjà une matrice de confusion, vous pouvez la refaire pour le meilleur modèle avec une visualisation améliorée.

In [ ]:
# Matrice de Confusion pour le Meilleur Modèle
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
ConfusionMatrixDisplay(conf_matrix, display_labels=encoder.classes_).plot(cmap='Blues', xticks_rotation=45)
plt.title("Matrice de Confusion pour le Meilleur Modèle")
plt.show()

## Étape 11 : Sauvegarder et Charger les Meilleurs Modèles

Pour réutiliser les meilleurs modèles sans avoir à les réentraîner, nous pouvons les sauvegarder et les charger ultérieurement.

### 11.1. Sauvegarde des Meilleurs Modèles

Les modèles sont déjà sauvegardés pendant l'entraînement grâce au callback `ModelCheckpoint`. Assurez-vous que les chemins des modèles sont correctement définis dans les résultats.

### 11.2. Chargement d'un Modèle Sauvegardé

Voici comment charger un modèle sauvegardé et l'utiliser pour faire des prédictions.

In [ ]:
# Exemple de chargement d'un modèle sauvegardé
best_result = results_df.sort_values(by='accuracy', ascending=False).iloc[0]
best_model_path = f"models/n_mfcc_{best_result['n_mfcc']}/{best_result['model_type']}_bs{best_result['batch_size']}_ep{best_result['epochs']}_lr{best_result['learning_rate']}.keras"
best_model = load_model(best_model_path)
print(f"\nChargement du meilleur modèle : {best_model_path}")

# Charger l'encodeur des labels (si non déjà chargé)
encoder = joblib.load('label_encoder.joblib')

### 11.3. Fonction de Prédiction

Voici une fonction de prédiction réutilisable avec le modèle chargé.

In [ ]:
def predict_genre(file_path, model, encoder, fixed_length=1293, n_mfcc=40):
    """
    Prédit le genre musical d'un fichier audio.
    """
    try:
        audio, sr = librosa.load(file_path, duration=30)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs = pad_or_truncate_mfcc(mfccs, fixed_length)
        mfccs = mfccs[..., np.newaxis]  # Ajouter la dimension canal
        mfccs = np.expand_dims(mfccs, axis=0)  # Ajouter la dimension batch

        # Normalisation
        mfccs = mfccs / np.max(mfccs)

        # Prédiction
        y_pred_proba = model.predict(mfccs)
        predicted_genre_index = np.argmax(y_pred_proba, axis=-1)
        predicted_genre = encoder.inverse_transform(predicted_genre_index)[0]
        return predicted_genre, y_pred_proba
    except Exception as e:
        print(f"Erreur lors de la prédiction : {e}")
        return None, None


## Étape 12 : Prédiction sur de Nouveaux Échantillons

Téléchargeons un nouvel échantillon audio depuis YouTube et effectuons une prédiction en utilisant le meilleur modèle.

In [ ]:
# Télécharger un fichier audio depuis YouTube
youtube_url = "https://youtu.be/tAGnKpE4NCI?si=6me8uc6lZW-LZLrA" 
download_youtube_audio(youtube_url, "test_audio.wav")

# Faire une prédiction
predicted_genre, y_pred_proba = predict_genre("test_audio.wav", best_model, encoder, fixed_length=best_result['n_mfcc'], n_mfcc=best_result['n_mfcc'])
print(f"Le genre prédit pour 'test_audio.wav' est : {predicted_genre}")

### 12.1. Afficher les Probabilités de Prédiction

Vous pouvez également afficher les probabilités associées à chaque classe.

In [ ]:
if y_pred_proba is not None:
    probabilities = y_pred_proba.flatten()
    genre_probabilities = dict(zip(encoder.classes_, probabilities))
    sorted_genres = sorted(genre_probabilities.items(), key=lambda x: x[1], reverse=True)

    print("\nProbabilités de prédiction par genre :")
    for genre, prob in sorted_genres:
        print(f"{genre}: {prob * 100:.2f}%")

    # Bar plot des probabilités
    plt.figure(figsize=(12, 6))
    sns.barplot(x=list(genre_probabilities.keys()), y=list(genre_probabilities.values()))
    plt.title("Probabilités de Prédiction par Genre")
    plt.xlabel("Genre")
    plt.ylabel("Probabilité")
    plt.xticks(rotation=45)
    plt.show()


## Conclusion

En suivant ce notebook structuré en blocs, vous pouvez systématiquement tester différentes combinaisons de modèles et d'hyperparamètres pour optimiser la classification des genres musicaux. Voici quelques points clés à retenir :

- **Organisation en Blocs** : Chaque étape du processus est clairement séparée, ce qui facilite la maintenance et l'expérimentation.
- **Automatisation des Tests** : L'utilisation de boucles imbriquées permet de tester efficacement toutes les combinaisons d'hyperparamètres.
- **Métriques Complètes** : En plus de la précision, le rappel et le score F1 fournissent une vue plus équilibrée des performances du modèle, surtout en cas de déséquilibre des classes.
- **Visualisation des Résultats** : Les graphiques générés permettent une analyse visuelle rapide des performances des différents modèles et hyperparamètres.
- **Sauvegarde des Modèles** : Grâce à `ModelCheckpoint`, les meilleurs modèles sont automatiquement sauvegardés, ce qui évite de perdre les performances optimales obtenues.
- **Réutilisation des Modèles** : Les modèles sauvegardés peuvent être facilement chargés pour des prédictions futures sans nécessiter un nouvel entraînement.